# Experiment with Vectorized drawing commands

* Bulk drawing commands with numpy arrays
* Built-in `line-profiler` support

In [ ]:
# the following code is necessary for MacOS users
import sys

if sys.platform == 'darwin':
    # Equivalent to `%gui osx`
    get_ipython().run_line_magic('gui', 'osx')

In [ ]:
import numpy as np

import py5

First, use numpy to do the calculations for a spiral

In [ ]:
INNER_RADIUS = 10
OUTER_RADIUS = 350
ROTATION_COUNT = 10
STEPS_PER_ROTATION = 500
STEPS = ROTATION_COUNT * STEPS_PER_ROTATION

# angles of all of the vertices in radians
angles = np.radians(np.arange(0, STEPS) % 360)

# radii of all of the vertices from the center
radii = np.linspace(INNER_RADIUS, OUTER_RADIUS, STEPS)

# x and y coordinates of vertices
xvals = radii * np.cos(angles)
yvals = radii * np.sin(angles)

# assemble x and y coordinates into one array
coordinates = np.stack([xvals, yvals], axis=1)

Sketch code for rotating spiral:

In [ ]:
def setup():
    py5.size(750, 750, py5.P2D)
    py5.stroke_weight(10)
    py5.no_fill()
    py5.get_surface().set_always_on_top(True)
    py5.text_align(py5.CENTER, py5.CENTER)
    py5.text_size(20)


def draw():
    py5.background("white")
    py5.translate(py5.width / 2, py5.height / 2)
    py5.rotate(py5.radians(2 * py5.frame_count))

    with py5.begin_shape():
        # *** bulk drawing command ***
        py5.vertices(coordinates)

In [ ]:
py5.run_sketch()

How fast is the call to `py5.vertices()`?

Let's use `line_profiler` to find out. No need to fuss with decorators, support for `line_profiler` is built in to the library.

In [ ]:
py5.profile_draw()

In [ ]:
py5.print_line_profiler_stats()

It's about 9.5 ms per call to `py5.vertices()`.

But how much does this matter? What if we used a `for` loop?

In [ ]:
def draw():
    py5.background("white")
    py5.translate(py5.width / 2, py5.height / 2)
    py5.rotate(py5.radians(5 * py5.frame_count))

    # with py5.begin_shape():
    #     # *** bulk drawing command ***
    #     py5.vertices(coordinates)

    with py5.begin_shape():
        # *** use `for` loop instead ***
        for i in range(STEPS):
            py5.vertex(coordinates[i, 0], coordinates[i, 1])

In [ ]:
py5.run_sketch()

In [ ]:
py5.profile_draw()

In [ ]:
py5.print_line_profiler_stats()

In [ ]:
(4112 + 388) * STEPS / 1e6

# Summary

* py5 has extra drawing methods to operate on numpy arrays
* built-in `line_profiler` support helps pinpoint performance problems